#### CREDITS:
  * https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed  

In [53]:
#import libraries
import pandas as pd
import numpy as np
import re

import tweepy
from tweepy import OAuthHandler 
from textblob import TextBlob

#for sentiment analyzing
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.metrics import f1_score
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re

In [11]:
#Cleaned Data Sets
#All the tweets were tokenized, had the punctuation removed, and has no stop words

#Bloomberg
bloomberg_0_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Bloomberg/clean_bloomberg_0_df.csv"
bloomberg_0_cleaned_df = pd.read_csv(bloomberg_0_cleaned_url, error_bad_lines=False)
bloomberg_1_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Bloomberg/clean_bloomberg_1_df.csv"
bloomberg_1_cleaned_df = pd.read_csv(bloomberg_1_cleaned_url, error_bad_lines=False)
bloomberg_2_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Bloomberg/clean_bloomberg_2_df.csv"
bloomberg_2_cleaned_df = pd.read_csv(bloomberg_2_cleaned_url, error_bad_lines=False)

#Warren
warren_0_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Warren/clean_warren_0_df.csv"
warren_0_cleaned_df = pd.read_csv(warren_0_cleaned_url, error_bad_lines=False)
warren_1_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Warren/clean_warren_1_df.csv"
warren_1_cleaned_df = pd.read_csv(warren_1_cleaned_url, error_bad_lines=False)
warren_2_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Warren/clean_warren_2_df.csv"
warren_2_cleaned_df = pd.read_csv(warren_2_cleaned_url, error_bad_lines=False)

#Sanders
sanders_0_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Sanders/clean_sanders_0_df.csv"
sanders_0_cleaned_df = pd.read_csv(sanders_0_cleaned_url, error_bad_lines=False)
sanders_1_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Sanders/clean_sanders_1_df.csv"
sanders_1_cleaned_df = pd.read_csv(sanders_1_cleaned_url, error_bad_lines=False)
sanders_2_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Sanders/clean_sanders_2_df.csv"
sanders_2_cleaned_df = pd.read_csv(sanders_2_cleaned_url, error_bad_lines=False)

#Biden
biden_0_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Biden/clean_biden_0_df.csv"
biden_0_cleaned_df = pd.read_csv(biden_0_cleaned_url, error_bad_lines=False)
biden_1_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Biden/clean_biden_0_df.csv"
biden_1_cleaned_df = pd.read_csv(biden_1_cleaned_url, error_bad_lines=False)
biden_2_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Biden/clean_biden_0_df.csv"
biden_2_cleaned_df = pd.read_csv(biden_2_cleaned_url, error_bad_lines=False)

#Trump
trump_0_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Trump/clean_trump_0_df.csv"
trump_0_cleaned_df = pd.read_csv(trump_0_cleaned_url, error_bad_lines=False)
trump_1_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Trump/clean_trump_1_df.csv"
trump_1_cleaned_df = pd.read_csv(trump_1_cleaned_url, error_bad_lines=False)
trump_2_cleaned_url = "https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Trump/clean_trump_2_df.csv"
trump_2_cleaned_df = pd.read_csv(trump_2_cleaned_url, error_bad_lines=False)

In [12]:
#gets the tweets you want to do the sentiment analysis on
def buildApplicableSet(df):
    testSet = []
    try: 
        for i in df["Text"].tolist():
            testSet.append({"text": i, "label": None})
        return testSet
    except:
        print("Unfortunately, something went wrong..")
        return None

In [13]:
# initialize api instance
TW_ACCESS_TOKEN_KEY = '1232422602146119682-Se6DgjBUx34m3bdewNNE9QUrH1AJ64'
TW_ACCESS_TOKEN_SECRET = 'ir24sl4sSaNmGhARRZrHxAUtKoQU0YecG9P9t1ckTRZOO'
TW_CONSUMER_KEY = '9M446vUPAwxFEINH5loMQ7ADU'
TW_CONSUMER_SECRET = 'a0H0OeyGGwV61puw0Fyjq06nEfzmMN6WmsBDM5nINO7nt5fuqj'

auth = tweepy.OAuthHandler(TW_CONSUMER_KEY, TW_CONSUMER_SECRET)
auth.set_access_token(TW_ACCESS_TOKEN_KEY, TW_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

#status = "Testing!"
#api.update_status(status=status)

In [14]:
#builds the training set for the naive bayes classifier
def buildTrainingSet(corpusFile, tweetDataFile):
    import csv
    import time
    
    corpus = []
    
    with open(corpusFile,'r') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
            
    rate_limit = 180
    sleep_time = 900/180
    
    trainingDataSet = []
    
    for tweet in corpus:
        try:
            status = api.get_status(tweet["tweet_id"])
            #print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(sleep_time) 
        except: 
            continue
            
    # now we write them to the empty CSV file
    with open(tweetDataFile,'w') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                 print(e)
    print("DONE")
    return trainingDataSet

In [15]:
#do NOT change the trainingData variable
#the cell takes around 4-5 hours to run

corpusFile = "corpus.csv"
tweetDataFile = "tweetDataFile.csv"

#trainingData = buildTrainingSet(corpusFile, tweetDataFile)

DONE


In [55]:
#trainingData

In [43]:
#training data for the classifier
trainingDataSet = trainingData[:2000]

In [44]:
#test data to test the accuracy of the classifier
testDataSet = trainingData[2000:]

In [60]:
#processes the tweets so they all have the same format
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        if type(tweet) == str or type(tweet) == bytes:
            tweet = tweet.lower() # convert text to lower-case
            #to remove links that start with HTTP/HTTPS in the tweet
            tweet = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)',
                   '', tweet, flags=re.MULTILINE) 
            #to remove other url links
            tweet = re.sub(r'[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', '', 
                        tweet, flags=re.MULTILINE)
            #remove punctuation
            punctuations = '''!()-[]{};:'"\,<>./?$%^&*_~'''
            tweet = ''.join([i for i in tweet if not i in punctuations])
            tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
            tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
            return [word for word in tweet if word not in self._stopwords]
        else:
            return[tweet]
    
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingDataSet)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

In [61]:
#Those word_features are basically a list of distinct words,
#each of which has its frequency (number of occurrences in the set) as a key.

import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [62]:
#True for ‘present’ and False for ‘absent’ if the word in tweet that is being tested
#is in training vocabulary

def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features 

In [66]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
testFeatures = nltk.classify.apply_features(extract_features, preprocessedTestSet)
testFeatures

[({'contains(get)': False, 'contains(swype)': False, 'contains(iphone)': False, 'contains(crack)': False, 'contains(hilarious)': False, 'contains(video)': False, 'contains(guy)': False, 'contains(duet)': False, 'contains(siri)': False, 'contains(pretty)': False, 'contains(much)': False, 'contains(sums)': False, 'contains(love)': False, 'contains(affair)': False, 'contains(httptco8exbnqjy)': False, 'contains(made)': False, 'contains(easy)': False, 'contains(switch)': False, 'contains(see)': False, 'contains(ya)': False, 'contains(16)': False, 'contains(strangest)': False, 'contains(things)': False, 'contains(said)': False, 'contains(far)': False, 'contains(sooo)': False, 'contains(glad)': False, 'contains(gave)': False, 'contains(sense)': False, 'contains(humor)': False, 'contains(httptcotwaeudbp)': False, 'contains(via)': False, 'contains(great)': False, 'contains(close)': False, 'contains(personal)': False, 'contains(event)': False, 'contains(tonight)': False, 'contains(regent)': Fals

In [64]:
#training the Naive Bayes classifier
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)
NBayesClassifier

In [67]:
#want to test the accuracy of the classifier
print("Classifier accuracy percent:",(nltk.classify.accuracy(NBayesClassifier, testFeatures))*100)
NBayesClassifier.show_most_informative_features(15)

#confusion matrix code

Classifier accuracy percent: 29.613733905579398
Most Informative Features
            contains(de) = True           irrele : neutra =     81.6 : 1.0
            contains(la) = True           irrele : neutra =     48.0 : 1.0
            contains(el) = True           irrele : neutra =     46.8 : 1.0
           contains(que) = True           irrele : neutra =     35.5 : 1.0
           contains(con) = True           irrele : neutra =     31.0 : 1.0
    contains(nexusprime) = True           irrele : positi =     28.9 : 1.0
      contains(sandwich) = True           positi : negati =     25.2 : 1.0
         contains(cream) = True           neutra : negati =     23.9 : 1.0
          contains(time) = True           negati : irrele =     23.6 : 1.0
           contains(ice) = True           neutra : negati =     23.5 : 1.0
          contains(para) = True           irrele : neutra =     23.1 : 1.0
        contains(thanks) = True           positi : irrele =     22.0 : 1.0
        contains(issues) =

In [62]:
#Application to Elections tweets
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")

Overall Negative Sentiment
Negative Sentiment Percentage = 10.76%
